# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [6]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(2)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        break

    last_height = new_height
    '''
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [8]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [9]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 金融海嘯以來最強一季！主計總處：Q3 GDP成長達3.33%　推算今年經濟成長1.9%　

▲行政院主計總處發布新數據，Q3經濟表現亮眼。（圖／記者吳佳穎攝）
記者吳佳穎／台北報導
行政院主計總處（30）日發佈今年第三季國民所得GDP概估統計，第三季GDP為3.33%，與8月預測的2.01%，增加了1.32個百分點。主計總處表示，由於出口優於預期，實值出口成長約8.4%，而國內消費也因振興券活絡，反彈可說是金榮海嘯以來最強勁的一季，若按原先估的第4季成長推算，全年GDP已達1.9%，但要再密切觀察疫情影響。
由於今年上半年經濟成長是0.78%，下半年第3季表現因出口優於預期、繳出亮眼成績單，所以原本預計的全年成長可能再上調至1.9%，但要看第4季疫情狀況再作調整。
主計總處表示，第三季外銷受惠5Ｇ布建、遠距商機及電子新品備貨需求，其中占比最高的電子零組件就增加了20.39%，資通與視聽產品也增加20.64%；不過金屬製品、機械及塑橡膠出口仍減8.18%、8.38%、4.65%，但減幅已較第二季縮小。
內需方面，由於國內疫情控制得宜，且振興券發揮功效，國人今年第三季在國內消費增加5.09%。雖然國籍航空收入銳減86.89%，台鐵、捷運及高鐵運量減幅近一成，但暑期旅遊旺季加上振興消費刺激及宅經濟挹注網購熱潮，零售業及餐飲營業額轉為正成長4.57%及0.85%，住宿消費減幅也在縮小。
另外，股市交易活絡上市櫃股票成交值增96.85%，加上自然人比重提高，帶動證券交易手續費大幅增加。
----------------------------------------------------------------------

[1] 康和證券歡慶30年　轉型創新超前布署迎接新挑戰

▲康和證券歷任董事長合照。右到左，葉公亮前董事長，鄭國華創辦人，鄭世華前董事長，鄭大宇董事長，周康記前董事長。（圖／康和提供）
記者陳心怡／台北報導
康和證券積極轉型創新、聚焦核心，在資訊科技與泛自營操作皆建立自有獨特的優勢，成立屆滿30週年公司經營成果不俗。康和證券集團董事長鄭大宇表示，康和證券沒有金控背景，依靠資源整合與超前布署來迎接市場挑戰

▲iPhone 12需求旺盛，但可能有4種隱憂。（圖／記者李毓康攝）
記者林妤柔／綜合報導
蘋果推出iPhone 12需求強勁，許多網友慘曝預定要等上一個月，連執行長庫克坦言有供貨吃緊的狀況。雖然iPhone 12熱銷，可望掀起自iPhone 6系列以來的史上最大換機潮，但用戶仍發現有些問題。
----------------------------------------------------------------------

[12] 各國央行拋售　WGC：Q3全球黃金市場需求降至11年來新低

▲2020年第3季的全球黃金需求降至11年來新低。（圖／視覺中國CFP）
記者林鼎閎／綜合報導
世界黃金協會（WGC）近日表示，2020年第3季的全球黃金需求年減19%，降至11年來低點，主因投資者購買黃金意願趨緩，以及各國央行10年來首次出現淨賣出黃金 。黃金價格截至今（30）日下午1點價格來到每盎司1,875.20美元，上漲+0.47%。
新冠肺炎肆虐使珠寶市場銷量崩跌，而珠寶通常是黄金需求的最大來源，但這需求被投資者對於黄金的避險需求所抵消。
根據WGC的數據，7月至9月這一季全球黃金需求為892.3噸，較2019年第3季下降19％，是自2009年第3季以來的最低的一次。
今年1-9月的需求為2,972.1噸，比2019年同期下降10％，也是自2009年以來的最低水準。
----------------------------------------------------------------------

[13] 搶攻遊戲主機市場！　宏碁遊戲取得PS5台灣代理權

▲PS5。（圖／記者周宸亘攝）
記者姚惠茹／台北報導
宏碁（2353）今（30）日宣布，旗下宏碁遊戲取得台灣銷售代理權，代理Sony Interactive Entertainment Taiwan Limited（台灣索尼互動娛樂股份有限公司）在台灣所發行及代理發行的產品，包含次世代主機PlayStation 5等遊戲硬體、軟體及版權授權產品與周邊設備的銷售業務。
宏碁遊戲董事長高樹國表示，宏碁為全球五大電腦品牌之一，自推出旗下電競電腦品牌後，從電競產品、國際賽事到服務平台，推出完整電競生態圈，提供玩家熱血遊戲體驗。
高樹國指出，宏碁子公司宏碁遊戲自今年11月起將與台灣索尼互動娛樂攜手合

▲東森全球新連鎖繼去年在好運來洲際宴展中心舉辦年會之後，2020年將移師高雄展覽館盛大舉辦4千人盛會。（圖／資料照，下同）
記者林璟倩／綜合報導
百業蕭條的2020年，東森全球新連鎖打出「一支手機就能創業」的強大吸引力，爆發性逆勢成長，不僅業績開紅盤創造破億佳績、經銷商夥伴激增至4萬人，幫助許多受新冠肺炎疫情衝擊的家庭，透過簡單的分享、轉換消費習慣的方式，輕鬆建立自己的事業創造被動收入。各方面成績亮眼的東森全球新連鎖將在12/5於高雄展覽館盛大舉辦年會，以「攜手東森、成就人生」為年度主題，表揚今年成績亮眼的經銷商夥伴，同時宣布新的年度方向展望未來，希望能夠邀請更多人一同打造美麗「森」活圈。
東森全球新連鎖於高雄舉辦的年會活動預計將開放4千名經銷商夥伴參與，此次活動為2020年相當罕見的單一企業大型聚會活動，對東森集團而言別具意義，因此當天預計邀請高雄市長陳其邁共同見證，東森全球新連鎖也將正式捐贈5輛高爾夫球車給高雄市立壽山動物園，讓行動不便、老弱婦孺都能輕鬆享受遊園樂趣，共同度過美好時光。
▲東森全球新連鎖直銷公司赴翠柏新村安養中心為90歲以上爺奶慶生，並捐贈東森農場嚴選台東池上米200包照顧長輩。
東森全球新連鎖成立短短3年即成為直銷界大黑馬，除了集結各行各業有志創富的經銷商夥伴投入拚出驚人業績、改善夥伴生活品質，承襲集團總裁王令麟「取之於社會、用之於社會」的理念，不忘投入公益活動，付出實際行動傳遞愛與溫暖，先前已大方捐出千包東森農場的池上新米助弱勢，陸續捐給光仁社會福利基金會等社福團體，並前往安養院陪伴銀髮長輩同歡，希望透過利他的無限力量創造社會共好新局勢。
▲東森全球新連鎖年會活動全程將由經銷商事業夥伴之一的綜藝鬼才沈玉琳歡樂主持。
年會活動全程將由經銷商事業夥伴之一的綜藝鬼才沈玉琳歡樂主持，當天也將有天王級神秘嘉賓現身，風光舉行正式加入東森全球新連鎖的簽約儀式，天王本人也將見證分享「東森活泉精華膠囊」的魅力！過去頻頻傳出逆齡保養奇蹟的活泉精華膠囊，除了王令麟本身試用後大讚之外，也獲得生技學者、東森自然美顧問團陳彥榮實驗後背書其修復力，年會上再由天王加持，強化人證、物證、科學做見證，勢必成為東森全球新連鎖新年度的拳頭商品，讓更多人感受活泉精華膠囊的魅力；當天也將公布年度新品「森美妍極地雪玫瑰系列」、「精油系列」即將隆重上市，並準備年會現場獨家限定的經銷商限量新

圖、文／鏡週刊
26年前陳小姐剛結婚，蜜月旅行一趟回來，身上只剩下幾萬元；4年後，兩個小孩陸續報到，雖是雙薪家庭，但手頭總是很緊，銀行也沒有積蓄。一天，她與先生約定，為了小孩未來教育著想，必須強迫自己儲蓄；但當時兩人收入不高，所以每人每月僅拿出3,000元，投入最單純的台股基金。
「每個月扣款3,000元其實無感，吃頓大餐、買件衣服或每天一杯咖啡就沒了；所以小錢投入市場最大好處，在不論市場狂風暴雨或豔陽高照，每天都睡得著。」陳小姐說。也因為如此，2000年網路泡沫急殺下來，她跟先生兩人才能穩住陣腳，持續扣款不受影響。
甚至到了金融海嘯那年，他們還有能力再加碼。「定時定額扣款第10年，遇到金融風暴，台股跌到3,000多點，當時我跟先生收入都有增加，手頭相對寬裕，就決定再各挑一檔台股基金，同樣每月扣3,000元，趁市場便宜多存一點。」她笑著展示成績單，「一晃眼過了11年，當初加碼扣的兩檔基金，現在又幫我們存下了150萬元。」
而自1998年與先生約定每月扣款3,000元，20多年後兩人究竟累積出多少錢？答案是將近500萬元！「先生在2006年網路下單盛行時，不小心下錯單、贖回了一次，當時獲利已超過7成、拿回55萬元後仍繼續扣款；但跟我從頭到尾沒設停利、一直扣下去相比，累積金額少很多。」
陳小姐攤開對帳單說，這段時間她共投入156萬元，最後滾出364萬元，報酬率達133％；但先生在「不小心」停利下，累積報酬率反而只有68％，滾出金額為131萬元。「其實，當年有些同事跟著我一起做定時定額，但都在網路泡沫、金融海嘯時陸續停扣，現在來看都非常可惜。」
----------------------------------------------------------------------

[36] 台股跌10餘點！蘋果財報「藏隱憂」　蘋概股表現溫吞

▲台股早盤微幅下跌。（圖／記者林敬旻攝）
記者林妤柔／綜合報導
美股週四因經濟數據轉佳、新刺激法案重啟談判，穩定投資人緊張情緒，四大指數全面收紅。台股今以12651.35點開出，下跌11.56點。截至9點12分，指數下跌23.57點。
蘋果公布最新會計年度財報，營收、獲利佳，但整體iPhone銷售下滑，尤其大中華地區營收降低29%，導致盤後股價重挫逾5%，恐不利台蘋概股表現。
今蘋概三王表現溫吞，台積電（2330）今日以437

▲群創。（圖／記者姚惠茹攝）
記者姚惠茹／台北報導
群創（3481）今（29）日公布，第3季營收746億元，營業利益13億元，稅後淨利15億元，基本每股盈餘（EPS）0.16元，擺脫連續7季虧損，單季轉虧為盈，並看好旺季備貨需求強勁，造成上游供應鏈產能緊張，第4季面板價格漲勢將可延續。
----------------------------------------------------------------------

[48] 宏碁VitaBeauty搶攻美麗商機！　全新膠原蛋白、葉黃素飲品開賣

▲宏碁董事長陳俊聖手持「VitaBeauty HA」與「VitaBeauty Lutein 葉黃素飲」。（圖／記者姚惠茹攝）
記者姚惠茹／台北報導　
宏碁（2353）跨足生活風格品牌，繼日前推出PredatorShot能量飲料及VitaBeauty莓果膠原蛋白飲後，今（29）日宣布加碼推出全新「VitaBeauty HA」與「VitaBeauty Lutein 葉黃素飲」兩款飲品，並由旗下子公司展碁國際代理，明（30）日正式在Acer Store開賣。
宏碁表示，今年6月推出PredatorShot能量飲料及VitaBeauty莓果膠原蛋白飲後，市場反應熱烈，近期PredatorShot能量飲料更在全台全家便利商店及家樂福開賣，打造多元通路、搶攻電競玩家，因此加碼推出全新「VitaBeauty HA」與「VitaBeauty Lutein 葉黃素飲」兩款飲品。
宏碁指出，除了滿足電競玩家外，更著重愛美人士最在意的青春美麗與時尚需求，因此VitaBeauty品牌是以「美麗」作為品牌核心，包裝強調設計與時尚，顛覆以往相關產品主攻高齡市場的刻板印象，將健康、時尚、美麗，三者結合在一起的設計要素，成為產品賣點之一。
宏碁說明，全新「VitaBeauty HA」特別添加玻尿酸原料、5000mg加拿大極生膠原蛋白、鰹魚彈力蛋白肽，而且天然來源果汁萃取量高達45%，即使有少部分消費者較為抗拒的膠原蛋白腥味，也能以天然來源萃取果汁的甜味取代，8入盒裝訂價為960元。
宏碁補充，另一款VitaBeauty Lutein葉黃素飲，主要針對3C族，特別採用FloraGLO游離型葉黃素，其獨家小分子技術能使人體更快吸收，並添加紫蘿蔔、藍莓、黑醋栗、山桑子、雨生紅球藻（蝦紅素）、玉米黃素

▲財政部公布半導體機械設備出口情形，今年前9月已經取代工具機成為機械類最大出口品項。（圖／資料照）
記者吳靜君／台北報導
財政部今（29）日表示，隨著美、中、日、韓之間的政經爭端轉化為科技較勁，各國對自身科技力的提升益發著重，尤其在涉及利益最大的的半導體產業鏈。
財政部發布統計，我國的晶圓代工和封測領域技術在世界之首，近年來政府致力支持半導體產業及機械產業發展，加上國內半導體領頭企業也積極培養周邊配合的機械廠商自製能力，國內生產半導體等機械出口自2013年起已經連續八年創新高，今年前9月出口金額也達25億美元，占機械出口15.7%，九年之間上升10.2個百分點，擠下工具機成為機械類主要出口品項。
國內業者不斷研發新製程，也帶動生產半導體等機械進口遽增，今年前9月進口123億美元，占總機械進口五成三，進出口兩相比，國內半導體設備製造業仍有成長空間。
▲半導體機械設備進出口情形。（圖／財政部提供）
與世界各國相比，日本、美國及荷蘭是世界前三大生產半導體等機械出口的國家，出口市場集中在中國大陸、台灣和南韓，今年上半年三地出口值合計占世界比高達七成。
以荷蘭來說對中國大陸的出口已經達到第四位，主要是美國管制荷商出口紫外光光刻機（EUV）給陸商所導致；南韓因為國內半導體製造巨擘長期扶植該國產業鏈設備商，出口規模已經達到第五，產品也是近七成出口到中國大陸。
台灣半導體機械設備發展起步較晚，所以產製的設備外，出口以「中低階製程設備」及零件為主，主要出口對象為中國大陸占比42%、美國15%及日本11%。
至於全球前三大進口國依序為中國大陸、台灣及南韓，中國大陸為了提升半導體自製率，近年相關進口設備進口值高居全國首位，主要是日本、南韓進口，我國開發高階製程、擴大產能投資居次，南韓因為記憶體價格滑落，放緩投資步調，居第三。
▼世界主要半導體機械設備出口國加比較。（圖／財政部提供）
----------------------------------------------------------------------

[58] 理財周刊／布局第四季行情　怎麼抓潛力成長股？

文／理周教育學苑．方鈞
Q:全球股市在3月遭遇黑天鵝COVID-19疫情衝擊，道瓊指數跌了萬點，不斷上演熔斷，斷崖式的重挫，沒想到因居家隔離及發放失業紓困金，民眾手中有資金能流入股市，散戶成為第二季股市Ｖ轉主力

美中對抗以來，DRAM需求持續疲弱，原以為今年可伴隨５Ｇ發展恢復元氣，不料碰上疫情阻礙。好在產業終見落底訊號，相關廠商營運也將迎來回升態勢。
文／吳旻蓁
自一八年下半年的美中貿易戰開打後，動態隨機存取記憶體（DRAM）市場需求疲軟，導致嚴重供過於求，價格不斷走跌，一九年全年記憶體市場成長率大幅萎縮逾三成。原本市場期待今年隨著ＡＩ、５Ｇ等技術的日益成熟化，可為記憶體帶來許多創新應用，產業面將逐漸恢復元氣，然而突然爆發的新冠肺炎疫情與日漸升級的美中對抗卻打亂了記憶體的復甦腳步。
在今年第一季新冠肺炎大流行以前，IC Insights原預測今年整個記憶體市場將呈現十四％的強勁成長率，然而隨著大環境變化多端，記憶體成長率預估遭下修。雖今年上半年因肺炎衍生的科技商機，帶動ＮＢ、伺服器、資料中心等對於DRAM需求提升，加上宅經濟發威，促進電競產品、遊戲機銷售暢旺，均使DRAM出貨量成長，進而帶動價格上漲。
谷底已到 DRAM蓄勢反彈
但隨著時序進入下半年，因市場訂單已多於上半年完成備貨，加上華為禁令前的拉貨效應也於九月結束，使DRAM市況反轉，價格跌勢再起，預估第四季將再下跌約五％。不過，市場也看好，在廠商持續庫存去化，以及全球前三大DRAM廠包括三星、ＳＫ海力士及美光今年資本支出已轉為保守，預估明年上半年供給成長有限，DRAM將有望展現強勁成長態勢。
法人預測，因不少被遞延的３Ｃ需求將重新啟動，加上雲端新常態將使資料中心的落實成為近年DRAM需求成長的主要推手，明後年產業連續成長可期，其中，摩根大通看好，繼今年DRAM市場持平表現後，明年可望成長九％，達六八○億美元。在DRAM產業落底訊號浮現，外加美國有可能進一步對中國記憶體廠如長鑫存儲、長江存儲等祭出禁令，都將構成國內記憶體廠商營運與股價動能的一大助力，帶動包括南亞科、威剛、華邦電、旺宏、宇瞻、宜鼎、創見　等廠商營運見曙光。（全文未完）
全文及圖表請見《先探投資週刊2115期精彩當期內文轉載》
----------------------------------------------------------------------

[66] 滿額送按摩椅！他一看「消費滿100萬」嚇歪　網笑：其實很簡單

▲百貨公司時常推出促銷活動刺激消費。（示意圖／取自Pixabay）
記者曾筠淇／綜合報導
每逢週年慶，各大百貨公司都

▲低總價、高投報優勢，引爆北客搶購潮，周末的房產講座場場爆滿。
【企劃特輯】
全球央行印鈔救經濟，啟動貨幣寬鬆，產生通膨效應, 讓存在銀行的錢變得更薄，也象徵財富重分配時機來臨，許多高資產客群將資金從動蕩的股市及海外轉回台灣，看好台灣房地產具有保值、穩定等優勢，加上房貸利率探底讓成本大降，將熱錢投入房市成為資產配置最佳首選，此現象也反映在今年北市豪宅市場上，許多置產投資需求的買盤也浮現，房市捷報頻傳。今年許多置產投資客將眼光放遠，將焦點放在房價基期仍低的高雄「亞洲新灣區」，位居亞灣區核心的「繽海」，建商推出3年包租15%方案，身受北客喜愛，引爆驚人買氣。
千億建設投入亞洲新灣區，全台補漲力道最強
積極轉型的亞灣區占地600多公頃，近年政府也大力推動9000億大南方開發政策，成為台商新首選，其中五大建設於2020即將全數完工，包括高雄展覽館、高雄市立總圖書館、輕軌第一階段、海洋文化及流行音樂中心，以及港埠旅運中心等落成，帶動房市熱度。建設到位吸引許多集團插旗投資，像是預計今年開幕的遠雄洲際飯店、海霸王集團85五星級飯店、日式精品飯店大倉久和，也預備在今年動工，未來更有夢時代二期、迪卡儂全台旗艦店等進駐，利多不斷，讓亞灣區躍升近二年高雄房價漲幅最大的區域之一。
▼亞洲新灣區匯集建設、交通、觀光等複合式優勢，勢必帶動房價增值。
高雄富豪區+地王商圈，稀有小坪數1-3房
亞灣區周圍百坪豪宅鄰立，「繽海」規劃1-3房小坪數，近擁三多大遠百商圈，規劃無邊際空中泳池、720坪空中花園，全齡休閒俱樂部，打造城市度假宅。七分鐘生活圈，捷運三多商圈站+旅運中心站，大遠百、新光三越、SOGO等百貨就在附近。最強地段力與時尚捷運商圈雙加持下，開出亮眼成績。
3年15%高投報　超越定存.海外不動產
知名建商愛山林建設以及郡都集團雙強聯手，為了讓年輕人也能輕鬆住最好地段，推出包租代管方案，3年15%的高投報率，房價不到台北市的一半，租金卻是台北市8折，投報率遠剩定存，享受增值，還賺到租金，讓許多北客捨棄海外不動產與大坪數豪宅，轉而投資該案。用低總價、高投報入主亞灣區+三多大遠百商圈，成為吸引客戶購屋的最強誘因。
▼3年有總價15%的高投報率，超越定存與海外不動產。
-----------------------------------------------------------------

▲台灣在IC設計、晶圓代工的領域相當專業。（示意圖／取自Pixabay）
記者曾筠淇／綜合報導
日本常被視為技術先進的國家，不論是工業還是基礎研究上，都有著一定的水平。一名網友先前就好奇，在基礎研究這塊，台灣遠遠不及日本，為何台灣在晶圓代工產業卻比較厲害呢？貼文曝光後，立刻引發熱議，網友紛紛推測，「日本不喜歡做代工」、「日本的更賺+1」。
▲原PO好奇，台灣的晶圓代工產業為何領先日本？（圖／翻攝自PTT）
貼文一出，有網友便以台灣的角度提出看法，「因為泡沫經濟時代人工太貴，台灣那時候相對便宜」、「台灣早期發展經濟，對於環保議題不是很重視」、「台灣一路走來都是代工，紡織代工、PC（個人電腦）代工、半導體代工」、「台灣就只是超級打工仔」、「高汙染，也只有台灣人不要命的搶著做」。
另外也有網友表示，「日本搞創新」、「日本不願犧牲環境」、「人家做更賺的」、「日本一堆專利」、「做研發居多」、「日本賣你原料就夠賺了好嗎」、「日本晶圓產能一點都不小，只是產值沒台灣那麼高」、「日本一堆專利幹嘛代工」、「代工高污染當然不做」、「台灣人努力在挖礦，日本就是賣鋤頭跟涼水給挖礦的人」。
----------------------------------------------------------------------

[86] 電子支付人氣旺！　產險業接力搶大餅

記者陳依旻／台北報導
電子支付已成國人行動支付新寵兒，富邦產險今（29）日宣佈攜手LINE Pay及一卡通，提供「即查即繳」及「掃描付款」繳付保險費服務。
顯示LINE Pay、街口支付成為國人行動支付的寵兒。國泰產險日前和街口支付合作加入電子支付可繳保費的行列，用戶可在街口支付的APP上自行選擇綁定信用卡享受街口幣回饋。另外， Apple Pay 跟 Google pay 也是大家常用的行動支付工具，新光產險開放街口支付繳保費的服務之外，也有與Apple Pay、Google pay 合作。
富邦產險指出，「即查即繳」主要服務投保強制險的機車族群，輸入簡易資料後即可查詢即將續保的強制險保單，提高機車險的續保率；「掃描付款」則提供給所有持有繳款單的保戶，透過LINE Pay條碼讀取器掃描QR Code的方式，再選擇LINE Pay Money帳戶付款，即可完成繳費。
富邦產險表示，民眾最常使用的繳費管道是超商繳款，今年